<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-2-Opti.-Mat./blob/main/P2_Tarea_2_Opti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install gurobipy

In [44]:
import numpy as np
from gurobipy import*

In [92]:
#mejor utilizamos librerias para resolver el problema xd:

betha = {
    0:0,
    1:100,
    2:120,
    3:130,
    4:140,
    5:150,
    6:200,
    7:0
} #betha es un vector columna

print(r[1])

c = {
    0:{1:10, 2:25, 3:30, 6:20},
    1:{4:15, 5:15},
    2:{5:15, 6:0},
    3:{6:10},
    4:{7:25},
    5:{7:80},
    6:{7:25},
    7:{}
}  # c es una matriz de 8x8


r = {
    0: {1: 500, 2: 350, 3: 500, 6: 600},
    1: {4: 250, 5: 350},
    2: {5: 350},
    3: {6: 500},
    4: {7: 800},
    5: {7: 500},
    6: {7: 500},
    7: {}
}  # r es una matriz de 8x8

n = 8


m = Model('Penalización')

#defino la varaible


x = m.addVars(n,n, vtype= GRB.CONTINUOUS,lb= 0, name = 'elemtos_por_bodega')

#añado las restricciones:


#hay que bloquear aquellos caminos donde r[i][j] == 0
for i in range(n):
  for j in range(n):
    if j not in r[i]:
      m.addConstr(x[i,j] == 0, name=f'Bloqueo_{i}_{j}')
    else:
      m.addConstr(x[i,j] <= r[i][j], name=f'Capacidad_{i}_{j}')


#Primera bodega tenga 1000

m.addConstr(quicksum(x[0, j] for j in range(n)) == 1000, name='Inicio')

#Ultima bodega tenga 1000

m.addConstr(quicksum(x[i,7] for i in range(n)) == 1000, name='Llegada')


#lo que entra por un nodo debe ser igual a lo que sale por este:

for i in range(1,7):
  entrada = quicksum(x[j,i] for j in range(n))
  salida = quicksum(x[i,j] for j in range(n))
  m.addConstr(entrada == salida, name=f'balance_{i}')

#definimos la función Objetivo:

costo_transporte = quicksum(c[i][j]*(x[i,j]*x[i,j]) for i in range(n) for j in range(n) if j in r[i])

costo_bodegaje = quicksum(betha[j]*(quicksum(x[k,j] for k in range(n))) for j in range(n) if j)

m.setObjective(costo_bodegaje + costo_transporte, GRB.MINIMIZE)

m.optimize()

if m.status == GRB.OPTIMAL:
    print("Solución óptima encontrada:")
    for i in range(n):
        for j in range(n):
            if x[i, j].x > 1e-6:  # Evitar imprimir valores cercanos a cero
                print(f'x[{i},{j}] = {x[i, j].x:.2f}')
else:
    print("No se encontró solución óptima")




{4: 250, 5: 350}
Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 72 rows, 64 columns and 164 nonzeros
Model fingerprint: 0xedd26eba
Model has 11 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 2e+02]
  QObjective range [2e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 65 rows and 53 columns
Presolve time: 0.01s
Presolved: 7 rows, 11 columns, 18 nonzeros
Presolved model has 11 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 7.000e+00
 Factor NZ  : 2.800e+01
 Factor Ops : 1.400e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 